Берем с помощью курса по сбору данных, селениума и старых галош комменты с сайта с фильмами к последниму сезону "Игры престолов" - чтоб было больше эмоций - собираем все в датасет.

In [1]:
import pandas as pd

df=pd.read_csv('coments2.csv')
df.head()

,index,content
0,0,NaN
1,1,NaN
2,2,"Ещё не смотрел, всё ждал когда все серии выйду..."
3,3,"Да во времена старой республики. Где то читал,..."
4,4,Он мог его зубами разорвать))


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3407 entries, 0 to 3406
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    3407 non-null   int64 
 1   content  3218 non-null   object
dtypes: int64(1), object(1)
memory usage: 53.4+ KB


Подготавливаем данные к обработке

In [3]:
df['content']=df['content'].fillna('не знаю')

In [4]:

df=df.drop('index',axis=1)

In [5]:
df.head()


,content
0,не знаю
1,не знаю
2,"Ещё не смотрел, всё ждал когда все серии выйду..."
3,"Да во времена старой республики. Где то читал,..."
4,Он мог его зубами разорвать))


Применяем курс по преносу обучения - берем с https://huggingface.co/ нейросеть seara/rubert-tiny2-russian-sentiment - для оценки комментов: neutral: нейтральный positive: позитивный negative: негативный.


In [6]:
from transformers import pipeline
from transformers import BertTokenizer
model = pipeline(model="seara/rubert-tiny2-russian-sentiment")

Собираем ф-ию приведения строки к числу для ускорения обработки. Ставим задачу оценить весь датасет комментов - положительный отрицательный или нейтральный

In [7]:
def estimateFunc(comment):
    mark={'positive':1,'negative':-1,'neutral':0}
    return mark[model(comment)[0]['label']]


Модель отказывалась работать без такой вот обрезки!

In [9]:
df['content'] = df['content'].str.slice(0, 2048)

Ф-ия рудуктора - просто складывает результаты выполнения map

In [10]:
from functools import reduce
def reduceButch(a,b):
    return a+b

Общая ф-ия

In [11]:
def butchState(butch):
    return reduce(reduceButch,map(estimateFunc,butch))

Посмотрим сколько у нас чего!

In [14]:
df['value']=df['content'].apply(estimateFunc)
df['value'].value_counts()

value
 0    2014
-1     991
 1     402
Name: count, dtype: int64

Много 0, но "-" больше чем "+"!

И контрольный на время

In [15]:
%%time
butchState(df['content'])

CPU times: total: 35.3 s
Wall time: 51.8 s


-589

Обработка всего цельного датасета заняла вот столько времени

Обработаем датасет кусками

In [16]:
%%time
chunksize = 200
temp=[]
for chunk in pd.read_csv('coments2.csv', chunksize=200):
    chunk['content']=chunk['content'].fillna('не знаю')
    chunk['content'] = chunk['content'].str.slice(0, 2048)
    # print(chunk)
    temp.append(butchState(chunk['content']))
    print(temp)

print(temp)
print(reduce(reduceButch,temp))

[25]
[25, -16]
[25, -16, -36]
[25, -16, -36, -32]
[25, -16, -36, -32, -55]
[25, -16, -36, -32, -55, -47]
[25, -16, -36, -32, -55, -47, -48]
[25, -16, -36, -32, -55, -47, -48, -26]
[25, -16, -36, -32, -55, -47, -48, -26, -44]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41, -32]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41, -32, -33]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41, -32, -33, -43]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41, -32, -33, -43, -44]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41, -32, -33, -43, -44, -38]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41, -32, -33, -43, -44, -38, 2]
[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41, -32, -33, -43, -44, -38, 2]
-589
CPU times: total: 34.4 s
Wall time: 53.4 s


А теперь распараллелим 

In [24]:
%%time
from joblib import Parallel, delayed
chunks = pd.read_csv('coments2.csv', chunksize=chunksize)
temp = Parallel(n_jobs=4)(delayed(butchState)(chunk['content'].fillna('не знаю').str.slice(0, 2048)) for chunk in chunks)
print(temp)
result = reduce(reduceButch, temp)
print(result)

[25, -16, -36, -32, -55, -47, -48, -26, -44, -36, -45, -41, -32, -33, -43, -44, -38, 2]
-589
CPU times: total: 1.77 s
Wall time: 22.4 s


Наблюдаем улучшение - время обработки сократилось! на 4 лучше чем на 16!

Испытаем еще один механизм распараллеливания

In [20]:
%%time

import threading
import queue

temp=[]
threads=[]
def butchState2(butch,result_queue):
    result_queue.put(reduce(reduceButch,map(estimateFunc,butch)))  # Сохраняем результат в очередь

chunksize = 200
result_queue = queue.Queue()

for chunk in pd.read_csv('coments2.csv', chunksize=chunksize):
    chunk['content']=chunk['content'].fillna('не знаю')
    chunk['content'] = chunk['content'].str.slice(0, 2048)
    threads.append(threading.Thread(target=butchState2,args=(chunk['content'],result_queue)))

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

temp = []
while not result_queue.empty():
    temp.append(result_queue.get())

print('All Done')
print(temp)
print(reduce(reduceButch,temp))
    

All Done
[2, -43, -32, -16, 25, -47, -33, -32, -55, -36, -44, -38, -48, -26, -36, -45, -44, -41]
-589
CPU times: total: 6.25 s
Wall time: 21.7 s


Не хуже предыдущего! Тут как бы все куски обрабатываются в отдельном потоке! Даже порядок результатов разный - что говорит о том что каждый поток завершается когда ему вздумается! И тут не с чем играться - нет возможности задать количество одновременно работающих потоков - все задается очередью автоматически.

Последний рабочий вариант по совету друзей! 

In [22]:
%%time
from concurrent.futures import ThreadPoolExecutor, as_completed

chunksize = 200
temp = []

with ThreadPoolExecutor(max_workers=4) as executor:  # Ограничиваем количество потоков
    futures = []
    for chunk in pd.read_csv('coments2.csv', chunksize=chunksize):
        chunk['content'] = chunk['content'].fillna('не знаю')
        chunk['content'] = chunk['content'].str.slice(0, 2048)
        futures.append(executor.submit(butchState, chunk['content']))

    for future in as_completed(futures):
        temp.append(future.result())  # Сбор результатов

final_result = reduce(reduceButch, temp)
print('All Done')
print(temp)
print(final_result)


All Done
[-32, -36, 25, -16, -55, -47, -26, -48, -36, -44, -45, -41, -32, -33, 2, -43, -44, -38]
-589
CPU times: total: 5.73 s
Wall time: 24.6 s


Результат средний! Можно играться с количеством воркеров. Интересно, что Wall time на 16 и на 4 одинаковый, а время цпу на 4 - меньше!

Тут процесс не пошел!

In [ ]:
# from multiprocessing import Pool


# # Главная функция для параллельной обработки файла
# def process_large_file_in_parallel(file_path, chunksize=10, num_workers=4):
#     pool = Pool(num_workers)  # Создаем пул процессов

#     # Читаем файл по частям и передаем части в пул для параллельной обработки
#     results = []
#     for chunk in pd.read_csv(file_path, chunksize=chunksize):
#         chunk['content'] = chunk['content'].fillna('не знаю')
#         chunk['content'] = chunk['content'].str.slice(0, 2048)
#         result = pool.apply_async(butchState, args=(chunk['content'],))
#         results.append(result)

#     pool.close()  # Закрываем пул, больше не принимаем задачи
#     pool.join()  # Ждем завершения всех процессов

#     # Собираем результаты
#     print(reduce(reduceButch,results))

# # Пример использования
# file_path = 'coments2.csv'
# process_large_file_in_parallel(file_path)
